In [1]:
import re


from sqlalchemy import create_engine, select
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.exc import DataError
from sqlalchemy.sql.sqltypes import SMALLINT

from event_generator import event_generator

In [2]:
renamed_keys = {'id': 'global_event_id', 'eventId': 'local_event_id','relatedEventId':'related_local_event_id'}

modified_keys = {'cardType', 'outcomeType', 'period', 'time', 'type'}

parsed_keys = {'globalGameId','teamId','playerId', 'relatedPlayerId', 'field','isShot', 'isTouch', 'isGoal', 'isOwnGoal','x', 'y','endX', 'endY', 'blockedX', 'blockedY', 'goalMouthY', 'goalMouthZ'}


value_quals = {30: 'involvedPlayers', 56: 'zoneGeneral', 130: 'teamFormation',
               131: 'teamPlayerFormation', 145: 'formationSlot', 194: 'captainPlayerId'}

bool_quals = {1: 'long_ball', 2: 'cross', 3: 'head_pass', 4: 'through_ball', 7: 'player_caught_offside',
              8: 'goal_disallowed', 13: 'foul', 14: 'last_man', 15: 'head', 29: 'assisted', 31: 'yellow_card',
              32: 'second_yellow_card', 33: 'red_card', 74: 'miss_high', 82: 'blocked', 88: 'high_claim',
              94: 'outfielder_block', 100: 'blocked_close', 101: 'saved_offLine', 154: 'intentional_assist',
              155: 'chipped', 156: 'lay_off', 169: 'leading_to_attempt', 170: 'leading_to_goal', 185: 'blocked_cross',
              190: 'from_shot_off_target', 210: 'shot_assist', 211: 'overrun', 214: 'big_chance', 
              241: 'indirect_freekick_taken', 242: 'obstruction', 264: 'aerial_foul', 285: 'defensive', 286: 'offensive',
              11111: 'intentional_goal_assist', 11112: 'big_chance_created', 11113: 'key_pass', 11114: 'void_yellow_card'}

lists = [({15, 20, 21, 72, 182, 183}, 'body_part'),
         ({9, 22, 23, 24, 25, 26, 160}, 'shot_situation'),
         ({73, 74, 75, 76, 77, 78, 79, 80, 81}, 'direction'),
         ({16, 17, 18, 19, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71}, 'zone'),
         ({5, 6, 107, 123, 124}, 'pass_situation'),
         ({173,174,177}, 'gk_save_result'),
         ({178,179}, 'gk_save_move'),
         ({186, 187, 188}, 'gk_penalty_result'),
         ({1115,11116,11117}, 'gk_save_location'),]

ignored_keys = {'expandedMinute','second','qualifiers','minute','satisfiedeventstypes','$idx','$len','minuteinfo','satisfiers','text'}
ignored_quals = {28,55,102,103,140,141,146,147,212,213,233}

In [3]:
def process_record(record):
    
    event = {}
    
    for k,v in renamed_keys.items():
        val = record.get(k)
        event[v] =  val
    
    for k in parsed_keys:
        event[k] = record.get(k)
        
    expandedMinute = record.get('expandedMinute')
    second = record.get('second')
    if None not in (expandedMinute,second):
        event['time'] = expandedMinute * 60 + second
    else:
        event['time'] = -1
        raise ValueError
    if event['time'] > 10000:
        event['time'] = -1
        
    event['type'] = record['type']['value']
    event['period'] = record['period']['value']
    
    outcome_type = record.get('outcomeType')
    card_type = record.get('cardType')
    event['outcome_type'] = outcome_type.get('value') if outcome_type else None
    event['card_type'] = card_type.get('value') if card_type else None
    
    for qual in record.get('qualifiers'):
        qual_type = qual['type']['value']
        
        if qual_type in value_quals:
            event[value_quals[qual_type]] = qual['value']

        elif qual_type in bool_quals:
            event[bool_quals[qual_type]] = True
            
        else:
            for lst in lists:
                if qual_type in lst[0]:
                    event[lst[1]] = qual_type
                    break
    for attr in ["field","local_event_id","period","type"]:
        if event.get(attr):
            if event[attr] > 3000 or event[attr] < 0:
                print(attr, event[attr])
                print(f'{event["globalGameId"]}')
                event[attr] =  -1
    if event['time'] is None:
        print(event)
        raise ValueError
    return event

In [4]:
def transfer_card_to_foul(card, foul, excluded_events):
    transferred_attrs = {'card_type','red_card','second_yellow_card','void_yellow_card','yellow_card'}
    if card.time - foul.time > 20 or card.player_id != foul.player_id:
        return
    for attr in transferred_attrs:
        setattr(foul, attr, getattr(card, attr))
    excluded_events.add(card.global_event_id)

def process_game(events):
    # print(f'processing game {events[0].global_game_id}, with {len(events)} events passed')
    excluded_events = set()
    foul_event_pair = None
    corner_awarded_pair = None
    start_pair = []
    first_action_found = None
    pending_card_event = None
    foul_commited_event = None
    
    sorted_events = sorted(events, key=lambda event: event.time)
    for e,event in enumerate(sorted_events):
        
        
        if event.type == 32: #Start
            if not start_pair:
                start_pair = [event]
            else:
                start_pair.append(event)
                
        elif not first_action_found and event.type not in [34] and len(start_pair) == 2:
            first_action_found = True 
            excluded_event = start_pair[0] if start_pair[0].team_id != event.team_id else start_pair[1]
            excluded_events.add(excluded_event.global_event_id)
            
        if event.type == 4: #Foul
            if not foul_event_pair:
                foul_event_pair = [event]
            else:
                foul_event_pair.append(event)

                foul_commited_event = foul_event_pair[foul_event_pair[0].outcome_type]
                foul_suffered_event = foul_event_pair[1 - foul_event_pair[0].outcome_type]
                setattr(foul_commited_event, 'relatedplayer_id', foul_suffered_event.player_id)
                excluded_events.add(foul_suffered_event.global_event_id)
                foul_event_pair = None
                
                if pending_card_event:
                    transfer_card_to_foul(pending_card_event, event, excluded_events)
                    
                
        elif event.type == 17:
            if not foul_commited_event:
                pending_card_event = event
                continue
            transfer_card_to_foul(event, foul_commited_event, excluded_events)
       
        elif event.type == 6: # Corner awarded
            if not corner_awarded_pair:
                corner_awarded_pair = [event]
            else:
                corner_awarded_pair.append(event)
                corner_awarded_event = corner_awarded_pair[1 - corner_awarded_pair[0].outcome_type]
                corner_suffered_event = corner_awarded_pair[corner_awarded_pair[0].outcome_type]
                setattr(corner_awarded_event, 'relatedplayer_id', corner_suffered_event.player_id)
                excluded_events.add(corner_suffered_event.global_event_id)
                corner_awarded_pair = None
               
    events = (event for event in events if event.global_event_id not in excluded_events)
    return events

In [1]:
engine = create_engine('postgresql://develop:}c%4Z>n~M<3p:Em\@localhost:5432/football')
Base = automap_base()
Base.prepare(autoload_with=engine, schema='core')
Event = Base.classes.events
regexp = re.compile(r'(?<!^)(?=[A-Z])')

NameError: name 'create_engine' is not defined

In [7]:
events = []
events_to_write = []
games_in_queue = 0

old_game_id = None

for record in event_generator(restricted_types={'OffsideGiven'}, shuffle=False):
    event = Event()
    rec = process_record(record)
    for k,v in rec.items():
        setattr(event,regexp.sub('_', k).lower(),v)
    
    if not old_game_id:
        old_game_id = event.global_game_id
        
    if event.global_game_id != old_game_id:
        events_to_write += process_game(events)
        games_in_queue += 1
        events = []
        if games_in_queue == 100:
            with Session(engine) as session:
                    session.add_all(events_to_write)
                    session.commit()
            events_to_write = []
            games_in_queue = 0
            print(f'batch inserted, last game_id = {old_game_id}')
    events.append(event)
    old_game_id = event.global_game_id
    
else:
    with Session(engine) as session:
            session.add_all(events)
            session.commit()
    print('remaining records inserted')

Event generator: started reading from 01.json
batch inserted, last game_id = 1115483
batch inserted, last game_id = 1187707
batch inserted, last game_id = 1187787
batch inserted, last game_id = 1187887
batch inserted, last game_id = 1190248
batch inserted, last game_id = 1190348
Event generator: finished reading from 01.json
Event generator: started reading from 02.json
batch inserted, last game_id = 1076326
batch inserted, last game_id = 1190528
batch inserted, last game_id = 1190803
batch inserted, last game_id = 1190884
batch inserted, last game_id = 1190984
batch inserted, last game_id = 1191064
Event generator: finished reading from 02.json
Event generator: started reading from 03.json
batch inserted, last game_id = 1201883
batch inserted, last game_id = 1201963
batch inserted, last game_id = 1202063
batch inserted, last game_id = 1221946
batch inserted, last game_id = 1222046
batch inserted, last game_id = 1080527
batch inserted, last game_id = 1222226
Event generator: finished r